In [1]:
from data_management.data_acquisition import DataAcquisition
from data_management.data_transformation import DataTransformation
from data_management import FROM_CSV
import pandas as pd
from datetime import timedelta
import seaborn as sns
from models.GCONVCheb import *
from trainer import Trainer
import matplotlib.pyplot as plt

/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
data_transform = DataTransformation(df, timedelta(365))
df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

2024-02-24 13:28:03.712 | INFO     | data_management._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


### Manual Elo with NN pass

In [3]:
# dummy dataset
from datetime import datetime
import numpy as np
import torch
from models.ratings.elo import *

delta = timedelta(days=365)
delta2 = timedelta(days=366)
now = datetime.now()
data = pd.DataFrame({'DT': [*(3 * [now]), *(3 * [now - delta2]), *(3 * [now - 2 * delta2]), *(3 * [now - 3 * delta2])], 
                     'Home': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                     'Away': ['B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B'],
                     'Winner': ['home', 'away', 'away', 'home', 'away', 'away', 'home', 'home', 'away', 'home', 'away', 'home'],
                     'Home_points': [10, 5, 15, 12, 15, 6, 20, 10, 10, 14, 3, 12],
                     'Away_points': [4, 8, 17, 10, 16, 14, 18, 9, 15, 0, 11, 4],
                     'League': [*(12 * ['liga'])],
                     })

transform2 = DataTransformation(data, timedelta(days=365))
dts = transform2.get_dataset(edge_f_one_hot=True)

def train_elo(train_dataset, model, epochs: int = 100, verbose: bool = False):
    torch.set_printoptions(precision=8) 
    training_accuracy = []
    loss_fn = torch.nn.MSELoss
    
    E_HS = []
    elos = []
    pts = []
    
    for epoch in range(epochs):
        model.train()
        accuracy, loss, count = 0, 0, 0
        iter = 0
        for time, snapshot in enumerate(train_dataset):
            # pass through network has to be only one by one in order to compute elo correctly
            matches = snapshot.edge_index
            match_points = snapshot.match_points
            for m in range(matches.shape[1]):
                match = matches[:, m]
                y_hat = model(match)
                y = np.array(snapshot.edge_attr[m, :])  # edge weight encodes the match outcome
                
                E_HS.append(y_hat)
                
                elos_b4 = (model.rating[model.home], model.rating[model.away])
                  
                target = np.argmax(y) / 2.
                prediction = y_hat
                accuracy += 1 if abs(target - prediction) < 0.33 else 0
    
                loss += np.mean((prediction-target)**2)
                
                point_diff = int(abs(match_points[m, 0] - match_points[m, 1]))
                result = [target, point_diff]
                
                model.backward(result)
                
                elos.append((elos_b4, (model.rating[model.home], model.rating[model.away])))
                pts.append(((match_points[m, 0], match_points[m, 1]), (model.home, model.away)))
                
                # if verbose:
                #    rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
                #    print(f"Neuralnet {iter}, rating: {rating}, E_H = {y_hat}; {target}")
                iter += 1
                
            count += matches.shape[1]

        if verbose:
            rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
            print(f'[TRN] '
                  f' Epoch: {epoch}, training loss: {loss:.3f}, '
                        f'training accuracy: {accuracy/count * 100:.2f}% \n'
                        f'ratings (first 5): {rating}')
        training_accuracy.append(accuracy/count * 100)
    
    return np.array(training_accuracy), E_HS, elos, pts


def compute_elo(matches: pd.DataFrame, mapping, elo_base: int = 1000, gamma: float = 2, c: float = 3, d: float = 500, k: float = 3, verbose: bool = False):
    elo = np.zeros((len(mapping), )) + elo_base
    E_HS = []
    elos = []
    for i in range(len(matches.index)):
        match_i = matches.iloc[i]
        h_i = mapping[match_i['Home']]
        a_i = mapping[match_i['Away']]
        
        E_h = 1 / (1 + np.power(c, ((elo[a_i] - elo[h_i]) / d)))
        S_h = 1. if match_i['Winner'] == 'home' else 0. if match_i['Winner'] == 'away' else 1/2
        
        E_HS.append(E_h)
        
        elos_b4 = (elo[h_i], elo[a_i])
        
        h_points = match_i['Home_points']
        a_points = match_i['Away_points']
        
        delta = abs(h_points - a_points)
        update = k * ((1 + delta)**gamma) * (S_h - E_h)
        
        elo[h_i] += update
        elo[a_i] -= update
        
        elos.append((elos_b4, (elo[h_i], elo[a_i])))
        
        if verbose:
            print(f"iteration {i}, rating: {elo}, E_H = {E_h}; {S_h}")
    return np.array(elo), E_HS, elos
    
    
print('Computed elo Dummy: ', compute_elo(data, transform2.team_mapping, verbose=False))
eloDummy = EloManual(team_count=transform2.num_teams, gamma=2., c=3., d=500., k=3.)
acc_dummy = train_elo(dts, eloDummy, 1, True)

print()
print('----------------------------------------')
print()
computed, E_HS_comp, elos_comp = compute_elo(df, transform.team_mapping, 1000, 2., 3., 500., 3.)

print()
elo = EloManual(team_count=transform.num_teams, gamma=2., c=3., d=500., k=3.)
acc_late, E_HS_net, elos_net, pts = train_elo(dataset, elo, 1, True)

err = False
for i in range(len(computed)):
    cmp = computed[transform.team_mapping[transform.inv_team_mapping[i]]]
    net = elo.rating[transform.team_mapping[transform.inv_team_mapping[i]]]
    if cmp != net:
        print(rf'ERROR on index {i}:: {cmp} / {net}')
        err = True
if not err:
    print("SUCCESS: Computed elo is the same as Elo from NN")


Computed elo Dummy:  (array([1450.41466185,  783.32864878,  766.25668937]), [0.5, 0.4597135195287363, 0.47177715569548395, 0.5421467296792649, 0.39493208333191193, 0.5532294506957484, 0.5379443202146263, 0.5865272540941598, 0.36183188188500987, 0.6768094454034466, 0.3183888246975012, 0.31720839917433186], [((1000.0, 1000.0), (1073.5, 926.5)), ((926.5, 1000.0), (904.4337510626207, 1022.0662489373793)), ((1022.0662489373793, 1073.5), (1009.3282657336013, 1086.237983203778)), ((1086.237983203778, 1009.3282657336013), (1098.6000215024378, 996.9662274349414)), ((904.4337510626207, 1098.6000215024378), (899.6945660626377, 1103.3392065024207)), ((996.9662274349414, 899.6945660626377), (862.5314709158746, 1034.1293225817046)), ((1103.3392065024207, 1034.1293225817046), (1115.8147098566258, 1021.6538192274995)), ((1021.6538192274995, 862.5314709158746), (1026.6154921783695, 857.5697979650045)), ((857.5697979650045, 1115.8147098566258), (818.4919547214234, 1154.892553100207)), ((1154.89255310020

### Elo with gradient